In [ ]:
from datetime import datetime
from pathlib import Path

import torch
import torchvision
import torchvision.transforms as transforms
from ada_verona import (
    AttackEstimationModule,
    AutoAttackWrapper,
    BinarySearchEpsilonValueEstimator,
    ExperimentRepository,
    ImageFileDataset,
    Network,
    One2AnyPropertyGenerator,
    PGDAttack,
    PredictionsBasedSampler,
    PytorchExperimentDataset,
    VerificationContext,
)
from autoverify.verifier import AbCrown
from robustness_experiment_box.verification_module.auto_verify_module import AutoVerifyModule

## Guide on how to compute upper bounds to Robustness Distributions using adversarial attacks with ada-verona. 

The notebook shows how to use the different components of [VERONA](https://github.com/ADA-research/VERONA) and the corresponding [ada-verona](https://pypi.org/project/ada-verona/) package for setting up robustness experiments and computing robustness distributions of neural networks [1,2]. 


We'll cover:
- Loading Models and Datasets with ada-verona
- Running PGD and AutoAttack experiments
- Running verification experiments with a formal verifier (AB-CROWN)

---
## 0. Setup

Make sure you have installed the ada-verona package from PyPI: `uv pip install ada-verona`. If you need further instructions, please refer to the [VERONA README](https://github.com/ADA-research/VERONA/blob/main/README.md).

To get started, you need a trained model and a dataset (e.g., MNIST).

## 1. Loading Models and Datasets with VERONA

The VERONA package is designed to make it easy for researchers to load models and datasets for robustness experiments.

## Supported Model Format
- **ONNX**: VERONA expects neural network models in the ONNX format (`.onnx` files).
- You can convert PyTorch or TensorFlow models to ONNX using their respective export utilities. 
- Note: We plan to add direct support for torch models soon (3. Quarter of 2025).
  


### Loading a Model

In [ ]:
# Path to your ONNX model
model_path = Path("data/MNIST/raw/models/mnist-net_256x2.onnx")
network = Network(model_path)

# To load as a PyTorch model (for attacks, etc.)
torch_model = network.load_pytorch_model()

### Loading a Dataset
- For standard datasets (like MNIST), use the provided wrappers:

In [ ]:
# define pytorch dataset. Preprocessing can be defined in the transform parameter
torch_dataset = torchvision.datasets.MNIST(
    root="./data", train=False, download=True, transform=transforms.ToTensor()
)
# wrap pytorch dataset into experiment dataset to keep track of image id
experiment_dataset = PytorchExperimentDataset(dataset=torch_dataset)

# work on subset of the dataset to keep experiment small
experiment_dataset = experiment_dataset.get_subset([x for x in range(0, 10)])

- For custom datasets (e.g., images on disk):

In [ ]:
image_folder = Path("path/to/images")
label_file = Path("path/to/labels.csv")
custom_dataset = ImageFileDataset(image_folder=image_folder, label_file=label_file)

### Sampling and Experiment Context
- Use the `ExperimentRepository` to manage experiments and create verification contexts. In this example, a one to any property generator is used that creates vnnlib files for one to any robustness queries. A one to one property generator is also already implemented in the package and could be used here as well. For the property generator, we have to define the number of classes, the lower bound of the data and the upper bound of the data.


In [ ]:
experiment_repository = ExperimentRepository(base_path=Path("experiments/"), network_folder=Path("models/"))
network = experiment_repository.get_network_list()[0]
data_point = experiment_dataset[0]
property_generator = One2AnyPropertyGenerator()
verification_context = experiment_repository.create_verification_context(network, data_point, property_generator)

**For more details, see the [VERONA wiki](https://deepwiki.com/ADA-research/VERONA).** 

## 2. Run PGD and AutoAttack Experiments 

Below is a minimal example for running PGD [3] and AutoAttack-based [4,5] robustness estimation.

- **Tip:** For more examples see the scripts in the `scripts/` folder.


In [ ]:
# Set up dataset and experiment repository
torch.manual_seed(0)
epsilon_list = [0.001, 0.005, 0.05, 0.08]
experiment_repository_path = Path("../tests/test_experiment")
network_folder = Path("data/MNIST/raw/models")

torch_dataset = torchvision.datasets.MNIST(
    root="./data", train=False, download=True, transform=transforms.ToTensor()
)
dataset = PytorchExperimentDataset(dataset=torch_dataset)
experiment_repository = ExperimentRepository(base_path=experiment_repository_path, network_folder=network_folder)
property_generator = One2AnyPropertyGenerator()

To compute the robustness of a network, one first has to check which data points are classified correctly. For that the PredictionsBasedSampler class is used.

In [ ]:
dataset_sampler = PredictionsBasedSampler(sample_correct_predictions=True)

# Here all the data points that are correctly predicted by the network are sampled
sampled_data = dataset_sampler.sample(network, experiment_dataset)

In [ ]:
# PGD Attack
verifier_pgd = AttackEstimationModule(attack=PGDAttack(number_iterations=10, step_size=0.01))
epsilon_value_estimator_pgd = BinarySearchEpsilonValueEstimator(
    epsilon_value_list=epsilon_list.copy(), verifier=verifier_pgd
)

# AutoAttack
verifier_autoattack = AttackEstimationModule(attack=AutoAttackWrapper())
epsilon_value_estimator_auto = BinarySearchEpsilonValueEstimator(
    epsilon_value_list=epsilon_list.copy(), verifier=verifier_autoattack
)

We use the pgd "verifier" `verifier_pgd ` from here on as an example for the next cells for illustrative purposes. Note that the verifier is used as a parameter to the Epsilon Estimator `epsilon_value_estimator_pgd`. The epsilon value estimator determines which search strategy is used to find adversarial examples (or, more precisely, estimate the interval of perturbation magnitude where the classification outcome changes). 

### Run the experiment (example for PGD)

Here is a minimal example for one network and one data point:

In [ ]:
network = experiment_repository.get_network_list()[0]
data_point = dataset[0]
verification_context = experiment_repository.create_verification_context(network, data_point, property_generator)
result = epsilon_value_estimator_pgd.compute_epsilon_value(verification_context)
print(result)

The slightly more elaborate example below computes critical epsilon values for a given network and datapoint and again defines a verification context. The folder for intermediate results needs to be provided to the VerificationContext, so the vnnlib files can be stored there. In addition, the results of the epsilon values queries can also be stored there.

In [ ]:
results = []
now = datetime.now()
now_string = now.strftime("%d-%m-%Y+%H_%M")

# Here the intermediate results (the per epsilon queries )
intermediate_result_base_path = Path(f"intermediate_results/{now_string}")

for data_point in sampled_data:
    network_name = network.path.name.split(".")[0]
    intermediate_result_path = Path(intermediate_result_base_path / f"{network_name}/image_{data_point.id}")

    verification_context = VerificationContext(
        network,
        data_point,
        intermediate_result_path,
        property_generator=property_generator,
    )
    epsilon_value_result = epsilon_value_estimator_pgd.compute_epsilon_value(verification_context)

    print(f"result: {epsilon_value_result}")
    results.append(epsilon_value_result)

## 3. Run AbCrown Verification Experiments

**Note:** Make sure that you have installed the `autoverify` package: `uv pip install autov-verify==0.1.4`.

Below is an example for running formal verification-based robustness estimation using the AbCrown verifier. This provides **lower bounds** to robustness distributions (as opposed to upper bounds from adversarial attacks).

- [**AB-CROWN**](https://github.com/Verified-Intelligence/alpha-beta-CROWN): A state-of-the-art neural network verifier that can provide formal guarantees


In [ ]:
# Set up experiment repository for AbCrown verification
experiment_name = "abcrown_verification"
timeout = 600  # 10 minutes timeout per verification query
experiment_repository_path = Path("../tests/test_experiment")
network_folder = Path("data/MNIST/raw/models")

# Initialize experiment repository
file_database = ExperimentRepository(base_path=experiment_repository_path, network_folder=network_folder)
file_database.initialize_new_experiment(experiment_name)

# Save experiment configuration
file_database.save_configuration(
    dict(
        experiment_name=experiment_name,
        experiment_repository_path=str(experiment_repository_path),
        network_folder=str(network_folder),
        timeout=timeout,
        epsilon_list=[str(x) for x in epsilon_list],
    )
)


In [ ]:
# Set up AbCrown verifier with AutoVerifyModule wrapper
property_generator = One2AnyPropertyGenerator()
verifier = AutoVerifyModule(verifier=AbCrown(), timeout=timeout)

# Create epsilon value estimator for formal verification
epsilon_value_estimator_abcrown = BinarySearchEpsilonValueEstimator(
    epsilon_value_list=epsilon_list.copy(), verifier=verifier
)

# Set up dataset sampler to only work with correctly predicted samples
dataset_sampler = PredictionsBasedSampler(sample_correct_predictions=True)


In [ ]:
# Run AbCrown verification experiment
network_list = file_database.get_network_list()

for network in network_list:
    print(f"Processing network: {network.path.name}")
    
    # Sample correctly predicted data points
    sampled_data = dataset_sampler.sample(network, dataset)
    print(f"Found {len(sampled_data)} correctly predicted samples")
    
    for data_point in sampled_data:
        # Create verification context for this network-data pair
        verification_context = file_database.create_verification_context(
            network, data_point, property_generator
        )
        
        # Compute epsilon value using formal verification
        epsilon_value_result = epsilon_value_estimator_abcrown.compute_epsilon_value(verification_context)
        
        # Save result to experiment repository
        file_database.save_result(epsilon_value_result)
        
        print(f"Result for image {data_point.id}: {epsilon_value_result}")

# Generate plots and save experiment results
file_database.save_plots()
print("AbCrown verification experiment completed!")


#### If you have questions or want to contribute, open an issue or pull request on our [VERONA GitHub](https://github.com/ADA-research/VERONA/issues)! Please also refer to our [VERONA wiki](https://deepwiki.com/ADA-research/VERONA) and more examples in the `scripts/` folder. 


### References:

[1] A. W. Bosman, H. H. Hoos, and J. N. van Rijn, “A Preliminary Study of Critical Robustness Distributions in Neural Network Verification,” Proceedings of the 6th workshop on formal methods for ML-enabled autonomous systems, 2023. Available: https://ada.liacs.leidenuniv.nl/papers/BosEtAl23.pdf


[2] A. Berger, N. Eberhardt, A. W. Bosman, H. Duwe, J. N. van Rijn, and H. Hoos, “Empirical Analysis of Upper Bounds of Robustness Distributions using Adversarial Attacks,” in THE 19TH LEARNING AND IN℡LIGENT OPTIMIZATION CONFERENCE, Accessed: Jul. 15, 2025. [Online]. Available: https://openreview.net/forum?id=jsfqoRrsjy

[3] A. Madry, A. Makelov, L. Schmidt, D. Tsipras, and A. Vladu, “Towards Deep Learning Models Resistant to Adversarial Attacks,” presented at the International Conference on Learning Representations, Feb. 2018. Accessed: Dec. 20, 2024. [Online]. Available: https://openreview.net/forum?id=rJzIBfZAb


[4] F. Croce and M. Hein, “Mind the box: l_1 -APGD for sparse adversarial attacks on image classifiers,” in International Conference on Machine Learning, PMLR, 2021, pp. 2201–2211. Accessed: Jul. 16, 2025. [Online]. Available: http://proceedings.mlr.press/v139/croce21a.html
[5] F. Croce and M. Hein, “Reliable evaluation of adversarial robustness with an ensemble of diverse parameter-free attacks,” in International conference on machine learning, PMLR, 2020, pp. 2206–2216. Accessed: May 03, 2025. [Online]. Available: https://proceedings.mlr.press/v119/croce20b.html

